In [1]:
import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np

In [2]:

label_path = 'data/article/網路危機_A1_文章危機程度.xlsx'
content_path = 'data/article/A1_article.xlsx'
label_df = pd.read_excel(label_path)

# read content from the worksheet "contents" in the excel file
content_df = pd.read_excel(content_path, sheet_name="contents")
# convert the content to string
label_df['TextID'] = label_df['TextID'].astype(str)
content_df['TextID'] = content_df['TextID'].astype(str)
print(label_df.shape)
print(content_df.shape)

(1424, 27)
(1424, 9)


In [2]:
labels = label_df['Crisis_Level'].tolist()
label_textid = label_df['TextID'].tolist()
dic = {}
for i in range(len(label_textid)):
    dic[label_textid[i]] = labels[i]

In [3]:
content_textid = content_df['TextID'].tolist()
content_label = []
for textid in content_textid:
    if textid not in dic:
        print(textid)
        print(type(textid))
    else:
        val = int(dic[textid])
        to_save = ''
        if val == 0:
            to_save = "d:0"
        elif val == 1:
            to_save = "a:A"
        elif val == 2:
            to_save = "b:B"
        else:
            to_save = "c:C"
        content_label.append(to_save)
content_df['Crisis_Level'] = content_label
# content_df.to_excel('data/article/A1_article_true_labeled.xlsx', index=False)

In [14]:
A1_article = pd.read_excel('data/article/A1_article_true_labeled.xlsx')
A2_article = pd.read_excel('data/article/A2_二次標註_網路危機訊息標註_1100927_僅全文標註(給致寧).xlsx')
A1_content = A1_article['Content(remove_tag)'].tolist()
A1_label = A1_article['Crisis_Level'].tolist()
A1_textid = A1_article['TextID'].tolist()
A1_title = A1_article['Title'].tolist()
A2_content = A2_article['Content(remove_tag)'].tolist()
A2_label = A2_article['Crisis_Level'].tolist()
A2_textid = A2_article['TextID'].tolist()  
A2_title = A2_article['Title'].tolist()
print(len(A1_content), len(A1_textid), len(A1_label), len(A2_content), len(A2_label), len(A2_textid))
A1A2_content = A1_content + A2_content
A1A2_label = A1_label + A2_label
A1A2_textid = A1_textid + A2_textid
A1A2_title = A1_title + A2_title    
print(len(A1A2_content), len(A1A2_label), len(A1A2_textid))
A1A2_df = pd.DataFrame()
A1A2_df['Content'] = A1A2_content
A1A2_df['Crisis_Level'] = A1A2_label
A1A2_df['TextID'] = A1A2_textid
A1A2_df['Title'] = A1A2_title


1424 1424 1424 1240 1240 1240
2664 2664 2664


In [15]:
print(A1_article.columns)
print(A2_article.columns)   

Index(['SerialID', 'TextID', 'Annotator', 'TextTime', 'Comment', 'Author',
       'Title', 'Content', 'Content(remove_tag)', 'Crisis_Level'],
      dtype='object')
Index(['SerialID', 'TextID', 'Crisis_Level', 'Author', 'Title',
       'Content(remove_tag)'],
      dtype='object')


In [16]:
# do one-hot encoding of "Crisis_Level"
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(A1A2_df[['Crisis_Level']])
# use the encoder to transform the label
one_hot = enc.transform(A1A2_df[['Crisis_Level']]).toarray()
add_columns = enc.categories_[0].tolist()
A1A2_df[add_columns] = one_hot

In [17]:
type1_1 = A1A2_df['a:A'].tolist()
# type1_0 is the complement of type1_1
type1_0 = []
for i in range(len(type1_1)):
    if type1_1[i] == 1:
        type1_0.append(0)
    else:
        type1_0.append(1)
# type2_1 is A1A2_df['b:B'] OR A1A2_df['a:A']
type2_1 = []
for i in range(len(type1_1)):
    if type1_1[i] == 1 or A1A2_df['b:B'][i] == 1:
        type2_1.append(1)
    else:
        type2_1.append(0)
# type2_0 is the complement of type2_1
type2_0 = []
for i in range(len(type2_1)):
    if type2_1[i] == 1:
        type2_0.append(0)
    else:
        type2_0.append(1)
# save
A1A2_df['type1_1'] = type1_1
A1A2_df['type1_0'] = type1_0
A1A2_df['type2_1'] = type2_1
A1A2_df['type2_0'] = type2_0

In [19]:
print(A1A2_df.head())

                                             Content Crisis_Level      TextID  \
0  以下文長...如果排版上有問題請見諒:loudly_crying_face:\n\n其實從小...          b:B  0038f0569e   
1  先不管之前我的朋友生了什麼病\n現在她都好了還硬要給這什麼預後的藥\n她說對成分會過敏也沒有...          d:0  003eb6998d   
2             幹你老師 雖小的一天，垃圾雙面心機婊滾去死吧，你就他媽的繼續演，看你能演多久          d:0  006da49ea0   
3  原po要軍種抽籤前一天晚上的課老師突然舉行考試來不及回家，因此只能請我媽代抽，結果好死不死直...          c:C  006f9d99fc   
4  不就工作嗎？為什麼都找不到，人生活著為什麼要工作，真的好想趕快死，因為我不敢自殺很痛，所以只...          b:B  007fb1eb08   

            Title  a:A  b:B  c:C  d:0  type1_1  type1_0  type2_1  type2_0  
0        憂鬱症的我...  0.0  1.0  0.0  0.0      0.0        1        1        0  
1            妳很過分  0.0  0.0  0.0  1.0      0.0        1        0        1  
2         bad day  0.0  0.0  0.0  1.0      0.0        1        0        1  
3  抽到海軍陸戰隊到親子關係破裂  0.0  0.0  1.0  0.0      0.0        1        0        1  
4   不就工作嗎（不喜請不要點開  0.0  1.0  0.0  0.0      0.0        1        1        0  


In [21]:
# save the dataframe to excel file
# A1A2_df.to_excel('data/article/A1A2_article.xlsx', index=False)
A1A2_df.to_excel('data/final/article/A1A2_article.xlsx', index=False)

In [22]:
print(A1A2_df.columns)

Index(['Content', 'Crisis_Level', 'TextID', 'Title', 'a:A', 'b:B', 'c:C',
       'd:0', 'type1_1', 'type1_0', 'type2_1', 'type2_0'],
      dtype='object')


In [5]:
# reduce the number of data by the ratio of 1:1:1:1
seed = 1
A1A2_df = pd.read_excel('data/final/article/A1A2_article.xlsx')
type_A_num = sum(A1A2_df['a:A'])
type_num = [type_A_num, 250, 250, 250]
all_kind_of_df = []
to_save_df = []
all_kind_of_df.append(A1A2_df[A1A2_df['a:A'] == 1])
all_kind_of_df.append(A1A2_df[A1A2_df['b:B'] == 1])
all_kind_of_df.append(A1A2_df[A1A2_df['c:C'] == 1])
all_kind_of_df.append(A1A2_df[A1A2_df['d:0'] == 1])
for i, a_df in enumerate(all_kind_of_df):
    # randomly choose type_A_num data from the dataframe
    a_df = a_df.sample(n=type_num[i], random_state=seed)
    to_save_df.append(a_df)
final_df = pd.concat(to_save_df)
# shuffle the dataframe
final_df = final_df.sample(frac=1, random_state=seed)
# show statistics
print(final_df.shape)
print(final_df['a:A'].sum())
print(final_df['b:B'].sum())
print(final_df['c:C'].sum())
print(final_df['d:0'].sum())
final_df.to_excel('data/final/article/A1A2_article_type3.xlsx', index=False)

(917, 12)
167
250
250
250


In [8]:
# reduce the number of data by the ratio of 1:1:1:1
seed = 1
A1A2_df = pd.read_excel('data/final/article/A1A2_article.xlsx')
type_A_num = sum(A1A2_df['type2_1'])
type_B_num = sum(A1A2_df['type2_0'])
print("type_A_num: ", type_A_num)
print("type_B_num: ", type_B_num)
type_num = [type_A_num, 500]
all_kind_of_df = []
to_save_df = []
all_kind_of_df.append(A1A2_df[A1A2_df['type2_1'] == 1])
all_kind_of_df.append(A1A2_df[A1A2_df['type2_0'] == 1])
for i, a_df in enumerate(all_kind_of_df):
    # randomly choose type_A_num data from the dataframe
    a_df = a_df.sample(n=type_num[i], random_state=seed)
    to_save_df.append(a_df)
final_df = pd.concat(to_save_df)
# shuffle the dataframe
final_df = final_df.sample(frac=1, random_state=seed)
# show statistics
print(final_df.shape)
print(final_df['type2_1'].sum())
print(final_df['type2_0'].sum())
final_df.to_excel('data/final/article/A1A2_article_type2.xlsx', index=False)

type_A_num:  485
type_B_num:  2179
(985, 12)
485
500


In [10]:
# reduce the number of data by the ratio of 1:1:1:1
seed = 1
A1A2_df = pd.read_excel('data/final/article/A1A2_article.xlsx')
type_A_num = sum(A1A2_df['type1_1'])
type_B_num = sum(A1A2_df['type1_0'])
print("type_1_num: ", type_A_num)
print("type_2_num: ", type_B_num)
type_num = [type_A_num, 200]
all_kind_of_df = []
to_save_df = []
all_kind_of_df.append(A1A2_df[A1A2_df['type1_1'] == 1])
all_kind_of_df.append(A1A2_df[A1A2_df['type1_0'] == 1])
for i, a_df in enumerate(all_kind_of_df):
    # randomly choose type_A_num data from the dataframe
    a_df = a_df.sample(n=type_num[i], random_state=seed)
    to_save_df.append(a_df)
final_df = pd.concat(to_save_df)
# shuffle the dataframe
final_df = final_df.sample(frac=1, random_state=seed)
# show statistics
print(final_df.shape)
print(final_df['type1_1'].sum())
print(final_df['type1_0'].sum())
final_df.to_excel('data/final/article/A1A2_article_type1.xlsx', index=False)

type_1_num:  167
type_2_num:  2497
(367, 12)
167
200
